In [ ]:
from collections import deque
from typing import List, Tuple, Set, Dict, Optional, NewType

In [ ]:
Board = List[List[int]]
Variable = Tuple[int, int]
Domains = Dict[Variable, Set[int]] 
Assignment = Dict[Variable, int] 

In [ ]:

class SudokuSolver:

    def __init__(self, board: Board):
        self.board = board
        self.variables: List[Variable] = [(r, c) for r in range(9) for c in range(9)]

        self.domains: Domains = {}
        for r in range(9):
            for c in range(9):
                var = (r, c)
                initial_value = board[r][c]
                # Set initial possibilities for each cell
                if initial_value == 0:
                    self.domains[var] = set(range(1, 10)) # Empty cell: 1-9
                elif 1 <= initial_value <= 9:
                    self.domains[var] = {initial_value} # Pre-filled cell: only its value
                else:
                    # Handle weird input values
                    print(f"Warning: Invalid value {initial_value} at {(r,c)}. Treating as empty.")
                    self.domains[var] = set(range(1, 10))

        # Find all neighbors for each cell (row, col, box)
        self.neighbors: Dict[Variable, Set[Variable]] = {var: self._get_neighbors(var) for var in self.variables}

        # Make sure initial clues are respected in domains
        if not self.enforce_node_consistency():
             print("Warning: Initial board seems inconsistent.")


    def _get_neighbors(self, var: Variable) -> Set[Variable]:
        """Find all cells constrained by this one (same row, col, or 3x3 box)."""
        r, c = var
        neighbors = set()
        # Row
        for col_idx in range(9):
            if col_idx != c:
                neighbors.add((r, col_idx))
        # Column
        for row_idx in range(9):
            if row_idx != r:
                neighbors.add((row_idx, c))
        # 3x3 Box
        start_row, start_col = (r // 3) * 3, (c // 3) * 3
        for row_idx in range(start_row, start_row + 3):
            for col_idx in range(start_col, start_col + 3):
                neighbor_var = (row_idx, col_idx)
                if neighbor_var != var:
                    neighbors.add(neighbor_var)
        return neighbors

    # 1. enforce_node_consistency
    def enforce_node_consistency(self) -> bool:
        """Ensure initial clues are the only option in their cell's domain."""
        for var in self.variables:
            r, c = var
            initial_value = self.board[r][c]

            if initial_value != 0: # If it's a pre-filled clue
                # Check if the clue is even possible (it should be)
                if initial_value not in self.domains[var]:
                     self.domains[var] = set() # Problem! Mark as impossible.
                     return False

                # Lock domain to the initial clue
                if self.domains[var] != {initial_value}:
                    self.domains[var] = {initial_value}

        return True # Looks consistent

    # 2. revise(x, y)
    def revise(self, x: Variable, y: Variable) -> bool:
        """Make cell 'x' consistent with cell 'y'.
           Remove values from x's domain if they have no possible match in y's domain.
           Returns True if x's domain was changed.
        """
        revised = False
        values_to_remove = set()

        for val_x in self.domains[x]:
            # Can val_x work with *any* value currently possible for y?
            if not any(val_x != val_y for val_y in self.domains[y]):
                # No value in y.domain allows val_x in x.domain
                values_to_remove.add(val_x)
                revised = True

        if revised:
            self.domains[x] -= values_to_remove

        return revised

    # 3. ac3()
    def ac3(self, initial_queue: Optional[List[Tuple[Variable, Variable]]] = None) -> bool:
        """Use AC-3 algorithm to make all cells arc-consistent.
           Returns False if an inconsistency (empty domain) is found.
        """
        # Start with all pairs of neighbors, unless a specific queue is given
        if initial_queue is None:
             queue = deque([(x, y) for x in self.variables for y in self.neighbors[x]])
        else:
             queue = deque(initial_queue)

        while queue:
            x, y = queue.popleft()

            if self.revise(x, y): # If we removed values from x...
                if not self.domains[x]:
                    return False 

                # Re-check neighbors of x (because x changed)
                for z in self.neighbors[x]:
                    if z != y:
                        queue.append((z, x))

        return True # Consistent

    # 4. assignment_complete(assignment)
    def assignment_complete(self, assignment: Assignment) -> bool:
         """Check if all cells have been assigned a value."""
         return len(assignment) == len(self.variables)

    def _is_value_consistent(self, var: Variable, value: int, assignment: Assignment) -> bool:
        """Check if this value works with the assigned neighbors."""
        for neighbor in self.neighbors[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False # Conflict!
        return True # OK with current assignments

   